# User Behavior Simulations with Gaussian Mixture Model assumptions

In this notebook we want to simulate different types of users by generating login timestamps after a Gaussian Mixture Model distribution. 

As a generating density, we will use a Gaussian mixture model of the form 

\begin{equation}
P(x | \Theta) = \sum\limits_{i=1}^n w_i \mathcal{N}(x | \theta_i) = \sum\limits_{i=1}^n w_i \mathcal{N}(\mu_i, \sigma_i^2)
\end{equation}

or a Gamma mixture 

\begin{equation}
P(x | \Theta) = \sum\limits_{i=1}^n w_i \mathrm{Gamma}(x | k_i, \theta_i),
\end{equation}

where the $w_i$ are weights chosen such that $\sum\limits_{i=1}^n w_i = 1$.

Different states of behavior can be simulated by adjusting the parameters of the distributions as well as the weights $w_i$. For instance, a more regular user will generally have sharper constituent distributions with small variance. This can also be the case if a user is constrained to a small window in his arrival time, which could happen e.g. due to a course that is offered at a gym at the same times on different days.

The scope of this notebook is to make a rapid prototype of user behavior simulation along with quick visualization to interpret the results. 

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

# Add directory above current directory to path
import sys; 
if not ".." in sys.path:
    sys.path.insert(0, '..')

%load_ext autoreload
%autoreload 2

import yaml

#cython support
import pyximport
pyximport.install(reload_support=True, language_level=2, setup_args={"include_dirs": np.get_include()})

#cython entropy functions, loaded separately
from regularity_analysis.regularity_measures import cython_approximate_entropy, cython_sample_entropy

from regularity_analysis.user_simulations import simulate_user_with_mixture, simulate_user_with_hmm
from regularity_analysis.feature_extraction import approximate_entropy, sample_entropy, permutation_entropy, integer_representation_entropy, fourier_spectrum_entropy, sliding_window_calculation
from utils.plot_utils import plot_user_histogram_with_density
from utils.animation_utils import *
from utils.time_utils import get_periodogram, get_welch_periodogram

In [2]:
#run this cell for fullscreen jupyter cells
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

We add a small plotting utility to overlay user histograms with their mixture densities.

Now we define a small routine that prints out our regularity statistics for a given user.

Now everything is set in place for the simulation to start. We define the parameters, mixture type and weights below.

In [3]:
#start simulating from January 1st, 2014
start_date = "01-01-2014"

#means and standard deviations
gaussian_means = np.array([2.0, 3.0])
gaussian_vars = np.array([0.01, 0.01])

gmm_weights = np.array([0.9, 0.1])
gmm_weights /= np.sum(gmm_weights)

#means and standard deviations, not equal to the actual gamma parameters!
gamma_means = np.array([2.0, 3.0, 5.0])
gamma_vars = np.array([0.1, 0.2, 0.3])

gamma_weights = np.array([0.8, 0.19, 0.01])
gamma_weights /= np.sum(gamma_weights)

In [4]:
user = simulate_user_with_mixture(start_date, gaussian_means, gaussian_vars, gmm_weights, num_samples=50)
fft_user = get_periodogram(user)
second_user = simulate_user_with_mixture(start_date, gamma_means, gamma_vars, gamma_weights, mixture_type="gamma", num_samples=200)
second_fft_user = get_periodogram(second_user)

# Part 2: User Simulations with Gaussian Hidden Markov Models

We just saw that by simply drawing a sample from the specified mixture distribution, the users we generate are localized in time, meaning that their timedelta distribution is rather sharp. This is to be expected, because we specified rather small variances in the simulation ($\sigma^2 ≤ 0.3$). 

However, when we calculate the regularity statistics for these users, we find that all of them produce comparatively high values, indicating that the users exhibit irregular behavior. This is affirmed by the generated time series plots, which show wildly oscillating spikes in between spaces of more regularity. This means that a localized timedelta distribution alone does not produce regular users, at least as long as there are enough different components with sufficiently high probabilities. 

The angle we take on regularity focuses on self-transitions in a simulated Markov process of the user. In dynamical systems theory, this is called \textit{recurrence}. Based on this concept, a user is regular if the probability of him staying in a given state is high. 

In the above example, all states are chosen based on weights $w_i$. States with higher weights are more common, while states with smaller weights are more rare. However, the transition probabilities of these states are equal, which means that each sample is drawn with the same fixed discrete probability distribution specified by the weight vector $w$. 

The Hidden Markov Model (HMM) models these transition probabilities for a set of finite symbols with a \textit{transition matrix} $A$. Each entry corresponds to the probability of transition from state i to state j in the next timestep, i.e.

\begin{equation}
A_{ij} = P(x_t = j | x_{t-1} = i). 
\end{equation}

The term "hidden" comes from the fact that these states cannot be directly measured, but rather a proxy variable that is tied to the hidden variable can be measured. In our setting, the state of mind or behavior of a human being is unobtainable, i.e. a hidden state, and we use timedelta data between visits to perform inference of behavioral state changes through changes in the observable. 

For this recipe to work, a probabilistic model of what state is most likely to emit what timedelta data has to be specified. For this, we use the \textit{emission matrix} $B$ whose entries are defined as 

\begin{equation}
B_{ij} = P(x_t = j | z_t = i). 
\end{equation}

Lastly, we need an initial distribution of observable states, commonly known as $\pi$. In our previous setting, this would correspond directly to the weight vector $w$. 

In [5]:
func_dispatch = {"apen": approximate_entropy, "sampen": sample_entropy, "entropy": fourier_spectrum_entropy}

In [6]:
@interact(m=(2,8,1), max_r=(0.2,3.0,0.1))
def apen_sampen_animation(m=2, max_r=1.0, N=50):
    """
    Small function for animating ApEn or SampEn as a function of the noise level r.
    Arguments:
    timeseries: pd.Series, holds values for simulation
    animation_config: dict, contains config for the animation
    ms: list, list of m parameters to use for ApEn and SampEn calculation
    rs: list, interval of rs to compute the ApEns and SampEns over 
    entropy_type: str, type of entropy to use
    """
    # set up figure and animation
    my_dpi = 220
    fig, ax = plt.subplots(figsize=(2560/my_dpi, 1600/my_dpi), dpi=my_dpi)
    
    reg_data = np.zeros((N,2))
    
    rs = np.linspace(0.01, max_r, num=N, endpoint=True)

    for i, r in enumerate(rs):
        reg_data[i,0] = approximate_entropy(user, m=m, r=r)
        reg_data[i,1] = sample_entropy(user, m=m, r=r)
    
    ax.grid()    
    ax.plot(rs, reg_data[:,0], rs, reg_data[:,1])

    ax.legend(['ApEn({}, {:0.2f})'.format(m, r), 'SampEn({}, {:0.2f})'.format(m, r)])
    ax.set_xlabel("r")
    ax.set_ylabel("ApEn/SampEn(m,r)")
    ax.set_xlim(rs[0], rs[-1])
    ax.set_ylim(0.0, 3.0)

    plt.show()

interactive(children=(IntSlider(value=2, description='m', max=8, min=2), FloatSlider(value=1.0, description='m…

In [8]:
@interact(m=(2,8,1), r=(0.2,3.0), n=(2,12,1))
def apen_fixed_animation(m=2, r=0.2, n=5, N=50):
    """
    Small function for animating ApEn or SampEn as a function of the noise level r.
    Arguments:
    timeseries: pd.Series, holds values for simulation
    animation_config: dict, contains config for the animation
    ms: list, list of m parameters to use for ApEn and SampEn calculation
    rs: list, interval of rs to compute the ApEns and SampEns over 
    entropy_type: str, type of entropy to use
    """

    # set up figure and animation
    my_dpi = 220
    fig, ax = plt.subplots(figsize=(2560/my_dpi, 1600/my_dpi), dpi=my_dpi)
    
    reg_data = np.zeros((N,3))
    
    running = np.arange(2,N+2)
    
    for i in running:
        user_values = np.zeros(i)
        user_values[0] = 1.0
        user_series = pd.Series(user_values)
        
        reg_data[i-2,0] = approximate_entropy(user_series, m=m, r=r)
        reg_data[i-2,1] = sample_entropy(user_series, m=m, r=r)
        reg_data[i-2,2] = integer_representation_entropy(user_series, n=n)
    
    ax.grid()
    
    ax.plot(running, reg_data[:,0], running, reg_data[:,1], running, reg_data[:,2])

    ax.legend(["ApEn({}, {:0.2f})".format(m, r), "SampEn({}, {:0.2f})".format(m, r), "IntRepEn({})".format(n)])
    ax.set_xlabel("Series Length")
    ax.set_ylabel("ApEn/SampEn(m,r)")
    ax.set_title("Entropies for a small number of visits")
    ax.set_xlim(running[0], running[-1])
    ax.set_ylim(0.0, 2.0)

    plt.show()

interactive(children=(IntSlider(value=2, description='m', max=8, min=2), FloatSlider(value=0.2, description='r…

In [9]:
@interact(mean=(1.0,10.0,0.1), m=(2,8,1), r=(0.2,3.0), N=(100,500,10), sigma=(0.001,5.0,0.1))
def single_user_sim(mean=5.0, sigma=0.01, entropy_type="ApEn", m=2, r=0.2, N=200):
    """
    Small function for animating a single Fourier spectrum entropy as a function of sigma in a user simulation.
    Arguments:
    animation_config: dict, contains config for the plot   
    mean: Mean of the Gaussian for timedelta simulation
    sigmas: tuple, range of sigmas to use in computation
    entropy_type: str, type of entropy to compute
    m: m used for ApEn and SampEn
    r: r used for ApEn and SampEn 
    """
    global func_dispatch
    my_dpi = 220
    # set up figure and animation
    fig = plt.figure(figsize=(2560/my_dpi, 1600/my_dpi), dpi=my_dpi)

    gs = fig.add_gridspec(2,3)
    ax1 = fig.add_subplot(gs[0,0])
    ax2 = fig.add_subplot(gs[0,1])
    ax3 = fig.add_subplot(gs[0,2])
    ax4 = fig.add_subplot(gs[1,:])

    key = entropy_type.lower()

    ts_entropy_names = {"apen": "Binary ApEn(m = {}, r = {})".format(m,r), "sampen": "Binary SampEn (m = {}, r = {})".format(m,r)}
    entropy_names = {"apen": "Power Spectrum ApEn(m = {}, r = {})".format(m,r), "sampen": "Power Spectrum SampEn (m = {}, r = {})".format(m,r), "entropy": "Power Spectrum Entropy"}
    kwds = {"apen": {"m": m, "r": r}, "sampen": {"m": m, "r": r}, "entropy": {"normalize": True}} 

    ax1.grid()
    ax2.grid()
    ax3.grid()
    ax4.grid()
    ax1.title.set_text("Timeseries")
    ax2.title.set_text("Density Histogram")
    ax3.title.set_text("FFT Power Spectrum")
    #ax4.legend()

    running = np.linspace(0.001, sigma, num=N, endpoint=True)

    reg_data = np.zeros((N, 3))

    ax4.set_xlabel("sigma^2")
    ax4.set_ylabel("Entropies")
    ax4.set_xlim(running[0], running[-1])
    ax4.set_ylim(0.0, 1.5)
    
    weights = np.ones(1)
    means = weights * mean
    variances = weights * sigma
    
    for i, sigma in enumerate(running):
        timeseries = simulate_user_with_mixture("01-01-2016", means=means, variances=variances, weights=weights, num_samples=100)
        bin_series = binary_user_from_td(timeseries)
        periodogram = get_periodogram(bin_series)
        if i == N - 1:
            bin_series.plot(ax=ax1, color="blue")
            plot_user_histogram_with_density(timeseries, means=means, variances=variances, weights=weights, ax=ax2)
            periodogram.plot(ax=ax3, color="blue")
        
        reg_data[i,0] = func_dispatch[key](bin_series, **kwds[key])
        reg_data[i,1] = func_dispatch[key](periodogram, **kwds[key])
        reg_data[i,2] = fourier_spectrum_entropy(periodogram)
        
    ax4.plot(running, reg_data[:,0], running, reg_data[:,1], running, reg_data[:,2])
    ax4.legend([ts_entropy_names[key], entropy_names[key], "Fourier Spectrum Entropy"])
    plt.show()

interactive(children=(FloatSlider(value=5.0, description='mean', max=10.0, min=1.0), FloatSlider(value=0.01, d…

In [10]:
from ipywidgets import FloatSlider, IntSlider, HBox, VBox
weight1_s = FloatSlider(min=0.0, max=1.0, step=0.01, value=0.5, description="weight1")
mean1_s = FloatSlider(min=1.0, max=50.0, step=1.0, value=2.0, description="mean1")
mean2_s = FloatSlider(min=1.0, max=50.0, step=1.0, value=7.0, description="mean2")
sigma1_s = FloatSlider(min=0.1, max=5.0, step=0.1, value=0.2, description="sigma1")
sigma2_s = FloatSlider(min=0.1, max=5.0, step=0.1, value=0.2, description="sigma2")
m_s = IntSlider(min=2, max=8, step=1, value=2, description="m")
r_s = FloatSlider(min=0.1, max=3.0, step=0.1, value=0.2, description="r")
N_s = IntSlider(min=100, max=500, step=10, value=200, description="N")

widget_list = [mean1_s, mean2_s, sigma1_s, sigma2_s, weight1_s, m_s, r_s, N_s]

widget_dict = {"mean1": mean1_s, "mean2": mean2_s, "sigma1": sigma1_s, "sigma2": sigma2_s, "weight1": weight1_s, "m": m_s, "r": r_s, "N": N_s}

def multi_user_sim(mean1, mean2, sigma1, sigma2, weight1, m, r, N, entropy_type="ApEn"):
    """
    Small function for animating a single Fourier spectrum entropy as a function of sigma in a user simulation.
    Arguments:
    animation_config: dict, contains config for the plot   
    mean: Mean of the Gaussian for timedelta simulation
    sigmas: tuple, range of sigmas to use in computation
    entropy_type: str, type of entropy to compute
    m: m used for ApEn and SampEn
    r: r used for ApEn and SampEn 
    """
    global func_dispatch
    my_dpi = 220
    # set up figure and animation
    fig = plt.figure(figsize=(2560/my_dpi, 1600/my_dpi), dpi=my_dpi)

    gs = fig.add_gridspec(2,3)
    ax1 = fig.add_subplot(gs[0,0])
    ax2 = fig.add_subplot(gs[0,1])
    ax3 = fig.add_subplot(gs[0,2])
    ax4 = fig.add_subplot(gs[1,:])

    key = entropy_type.lower()

    ts_entropy_names = {"apen": "Binary ApEn(m = {}, r = {})".format(m,r), "sampen": "Binary SampEn (m = {}, r = {})".format(m,r)}
    entropy_names = {"apen": "Power Spectrum ApEn(m = {}, r = {})".format(m,r), "sampen": "Power Spectrum SampEn (m = {}, r = {})".format(m,r), "entropy": "Power Spectrum Entropy"}
    kwds = {"apen": {"m": m, "r": r}, "sampen": {"m": m, "r": r}, "entropy": {"normalize": True}} 

    ax1.grid()
    ax2.grid()
    ax3.grid()
    ax4.grid()
    ax1.title.set_text("Timeseries")
    ax2.title.set_text("Density Histogram")
    ax3.title.set_text("FFT Power Spectrum")
    #ax4.legend()

    running = np.linspace(0.01, max(sigma1,sigma2), num=N, endpoint=True)

    reg_data = np.zeros((N, 3))

    ax4.set_xlabel("sigma^2")
    ax4.set_ylabel("Entropies")
    ax4.set_xlim(running[0], running[-1])
    ax4.set_ylim(0.0, 1.5)
    
    weights = np.array([weight1, 1.0-weight1])
    means = np.array([mean1, mean2])
    variances = np.array([sigma1, sigma2])
    
    for i, sigma in enumerate(running):
        timeseries = simulate_user_with_mixture("01-01-2016", means=means, variances=variances, weights=weights, num_samples=100)
        bin_series = binary_user_from_td(timeseries)
        periodogram = get_periodogram(bin_series)
        if i == N - 1:
            bin_series.plot(ax=ax1, color="blue")
            plot_user_histogram_with_density(timeseries, means=means, variances=variances, weights=weights, ax=ax2)
            periodogram.plot(ax=ax3, color="blue")
        
        reg_data[i,0] = func_dispatch[key](bin_series, **kwds[key])
        reg_data[i,1] = func_dispatch[key](periodogram, **kwds[key])
        reg_data[i,2] = fourier_spectrum_entropy(periodogram)
        
    ax4.plot(running, reg_data[:,0], running, reg_data[:,1], running, reg_data[:,2])
    ax4.legend([ts_entropy_names[key], entropy_names[key], "Fourier Spectrum Entropy"])
    plt.show()

left_box = VBox(widget_list[:4])
right_box = VBox(widget_list[4:])
ui = HBox([left_box, right_box])

out = widgets.interactive_output(multi_user_sim, widget_dict)

display(ui, out)

Output()

In [11]:
@interact(mean=(1.0,50.0,1.0), max_sigma=(0.01,5.0,0.1), N=(100,1000,10))
def fourier_spectrum_window_test(window1="boxcar", window2="hann", window3="parzen", mean=3.0, max_sigma=1.0, N=500):
    """
    Small function for animating the Fourier spectrum entropy as a function of sigma in a user simulation, with different window functions.
    Arguments:
    mean: Means of the Gaussians/Gammas for user simulation
    sigmas: tuple, range of sigmas to use in animation
    """
    means = np.array([mean])
    variances = np.array([max_sigma])
    weights = np.ones(1)

    my_dpi = 220
    # set up figure and animation
    fig = plt.figure(figsize=(2560/my_dpi, 1600/my_dpi), dpi=my_dpi)

    gs = fig.add_gridspec(2,2)
    ax1 = fig.add_subplot(gs[0,0])
    ax2 = fig.add_subplot(gs[0,1])
    ax3 = fig.add_subplot(gs[1,:])
    
    windows = list([window1, window2, window3])
    entropy_names = ["Power Spectrum Entropy, {} window ".format(window.capitalize()) for window in windows]
    
    running = np.linspace(0.001, max_sigma, num=N, endpoint=True)
    reg_data = np.zeros((N, 3))
    
    ax2.grid()
    ax3.grid()
    ax1.title.set_text("Density Histogram")
    ax2.title.set_text("FFT Power Spectrum")
    ax3.set_xlim(running[0], running[-1])
    ax3.set_ylim(0.0,1.0)
    ax3.set_xlabel("sigma^2")
    ax3.set_ylabel("Entropies")

    for i, sigma in enumerate(running):
        timeseries = simulate_user_with_mixture("01-01-2016", means=means, variances=variances, weights=weights, num_samples=N)
        bin_series = binary_user_from_td(timeseries)
        for k, window in enumerate(windows):
            periodogram = get_periodogram(bin_series, window=window)
            reg_data[i,k] = fourier_spectrum_entropy(periodogram)
            if i == N - 1:
                periodogram.plot(ax=ax2, label = entropy_names[k])
        if i == N - 1:
            plot_user_histogram_with_density(timeseries, means=means, variances=variances, weights=weights, ax=ax1)

    ax3.plot(running, reg_data[:,0], running, reg_data[:,1], running, reg_data[:,2])
    ax3.legend(entropy_names)            
    plt.show()

interactive(children=(Text(value='boxcar', description='window1'), Text(value='hann', description='window2'), …

In [12]:
@interact(mean=(1.0,50.0,1.0), sigma=(0.01,5.0,0.1), N=(100,1000,10))
def fourier_spectrum_winsize_test(mean=3.0, sigma=1.0, window1="boxcar", window2="hann", window3="parzen", max_window_size=125, N=300):
    """
    Small function for animating the Fourier spectrum entropy as a function of sigma in a user simulation, with different window sizes.
    Arguments:  
    mean: Means of the Gaussians/Gammas for user simulation
    sigmas: tuple, range of sigmas to use in animation
    """
    means = np.array([mean])
    variances = np.array([sigma])
    weights = np.ones(1)

    my_dpi = 220
    # set up figure and animation
    fig = plt.figure(figsize=(2560/my_dpi, 1600/my_dpi), dpi=my_dpi)

    gs = fig.add_gridspec(2,2)
    ax1 = fig.add_subplot(gs[0,0])
    ax2 = fig.add_subplot(gs[0,1])
    ax3 = fig.add_subplot(gs[1,:])

    windows = list([window1, window2, window3])
    entropy_names = ["Power Spectrum Entropy, {} window".format(window.capitalize()) for window in windows]
    
    ax3.grid()
    ax1.title.set_text("Timeseries")
    ax2.title.set_text("Periodogram")

    #reset the variances every few frames
    running = np.arange(1, max_window_size+1)

    reg_data = np.zeros((max_window_size, 3))
    
    timeseries = simulate_user_with_mixture("01-01-2016", means=means, variances=variances, weights=weights, num_samples=N)
    bin_series = binary_user_from_td(timeseries)
    
    plot_user_histogram_with_density(timeseries, means=means, variances=variances, weights=weights, ax=ax1)

    for k, window in enumerate(windows):
        reg_data[:,k] = np.array([fourier_spectrum_entropy(get_periodogram(bin_series, window=window, window_size=n)) for n in running])
        get_periodogram(bin_series, window=window, window_size=(k+1)*10).plot(ax=ax2, label = entropy_names[k] + ", len = {}".format(k*5))
    
    ax3.set_xlabel("Window Size N")
    ax3.set_ylabel("Entropies")
    ax3.set_xlim(running[0], running[-1])
    ax3.set_ylim(0.0, 1.0) #Fourier Spectrum Entropy is normalized

    ax2.grid()
    ax2.title.set_text("Periodogram")
    
    ax3.plot(running, reg_data[:,0], running, reg_data[:,1], running, reg_data[:,2])
    ax3.legend(entropy_names)            
    plt.show()

interactive(children=(FloatSlider(value=3.0, description='mean', max=50.0, min=1.0, step=1.0), FloatSlider(val…

In [9]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning) 
pattern = np.array([0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0])
@interact(reps=(5,500,5), pattern=fixed(pattern))
def fourier_spectrum_duration_test(pattern, window1="boxcar", window2="hann", window3="parzen", reps=100, nperseg=128, nfft=256, window_size=7):
    """
    Small function for animating the Fourier spectrum entropy of a completely regular customer.
    """

    my_dpi = 220
    # set up figure and animation
    fig = plt.figure(figsize=(2560/my_dpi, 1600/my_dpi), dpi=my_dpi)

    gs = fig.add_gridspec(2,2)
    ax1 = fig.add_subplot(gs[0,0])
    ax2 = fig.add_subplot(gs[0,1])
    ax3 = fig.add_subplot(gs[1,:])
    
    windows = [window1, window2, window3]
    num_windows = len(windows)

    entropy_names = ["Power Spectrum Entropy, {} window".format(window.capitalize()) for window in windows]
    welch_entropy_names = ["Welch Power Spectrum Entropy, {} window".format(window.capitalize()) for window in windows]

    names = entropy_names + welch_entropy_names

    cmap = plt.get_cmap("tab20", len(names))

    #reset the variances every few frames
    running = np.arange(1, reps + 1) * len(pattern)

    reg_data = np.zeros((len(running), 2 * num_windows))
    
    ax1.grid()
    ax2.grid()
    ax3.grid()
    ax1.title.set_text("Timeseries")
    ax2.title.set_text("Periodogram")
    ax3.set_xlim(running[0], running[-1])
    ax3.set_ylim(0.0, 1.0) #Fourier Spectrum Entropy is normalized
    ax3.set_xlabel("Sample Size N")
    ax3.set_ylabel("Entropies")
    ax3.set_xscale("log")

    for i, num in enumerate(running):
        vals = np.tile(pattern, i+1)
        timeseries = pd.Series(vals)
        
        if i == len(running) - 1:
            periodogram = get_periodogram(timeseries, window_size=window_size, fs=2.0)
            welch_periodogram = get_welch_periodogram(timeseries, window=window1, fs=2.0)
            timeseries.plot(ax=ax1, color="blue")
            periodogram.plot(ax=ax2, color="green")
            welch_periodogram.plot(ax=ax2, color="orange")
        for k, window in enumerate(windows):
            reg_data[i,k] = fourier_spectrum_entropy(get_periodogram(timeseries, window=window, window_size=window_size), normalize=True)
            reg_data[i,k+num_windows] = fourier_spectrum_entropy(get_welch_periodogram(timeseries, window=window, nfft=nfft, nperseg=nperseg), normalize=True)           
    
    ax3.plot(running, reg_data[:,0], running, reg_data[:,1], running, reg_data[:,2], running, reg_data[:,3], running, reg_data[:,4], running, reg_data[:,5])
    ax3.legend(entropy_names + welch_entropy_names)  
    plt.show()

interactive(children=(Text(value='boxcar', description='window1'), Text(value='hann', description='window2'), …